# Documentation ( More details coming soon)

How to use

* Add all .ser files in outputs folder. Note large .ser files takes a lot of time to load
* Run the corresponding cells.
* I have created few utitlities to show different variables stored in .ser files
* New to jupyter? See https://www.dataquest.io/blog/jupyter-notebook-tutorial/ .

In [ ]:
# Necessary import statements
!pip install javaobj-py3
import javaobj
import os

### Constants

In [2]:
root_path = '.'
output_folder = 'outputs'

### Helper Functions

In [3]:
def load_ser(path):
    with open(path, "rb") as fd:
        jobj = fd.read()
    return javaobj.loads(jobj)

# Left 1 ser file to compare? Simply call this function with correct file name to avoid 
def add_to_list(file_name,objects):
    file_names.append(file_name)
    print(f'Loading {item}')
    objects.append(load_ser(f'{root_path}/{output_folder}/{item}'))
    
def command_are_equal(a,b):
    return a.type==b.type and a.val == b.val and a.output == b.output and a.corr==b.corr and a.corr_addr == b.corr_addr and a.allocSize==b.allocSize and a.freeSize==b.freeSize


def compareTwo(obj1,obj2):
    if(len(obj1.testCases)!=len(obj2.testCases)):
        return False
    t = len(obj1.testCases)
    for i in range(t):
        comm1 = obj1.testCases[i].commands
        comm2 = obj2.testCases[i].commands
        if(len(comm1)!=len(comm2)):
            print(i)
            print(len(comm1))
            print(len(comm2))
            return False
        n_comm = len(comm1)
        
        for j in range(n_comm):
            if(not command_are_equal(comm1[j],comm2[j])):
                print(i,j,comm1[j].type,comm2[j].type,comm1[j].output,comm2[j].output)
                return i,j,comm1[j],comm2[j]
    return True

def is_in_comm_ops(arr,i):
    for item in arr:
        if(i in item):
            return True
    return False

## Load all .ser files in output folder

In [4]:
file_names = []
objects = []
for item in os.listdir(f'{root_path}/{output_folder}'):
    file_names.append(item)
    print(f'Loading {item}')
    objects.append(load_ser(f'{root_path}/{output_folder}/{item}'))

Loading pranjal_output_small_120411.ser


# Compare all outputs

In [24]:
comm_ops = [[]]
row_num  = -1
for k in range(len(objects)):
    if(is_in_comm_ops(comm_ops,k)):
        continue
    row_num = row_num+1
    if(row_num>0):
        comm_ops.append([])
    comm_ops[row_num] = [k]
    for i in range(k+1,len(objects),1):
        print(k,i)
        outp = compareTwo(objects[k],objects[i])
        if(outp == True):
            comm_ops[row_num].append(i)
            

# PRINT SAME OUTPUTS

In [25]:
for i,item in enumerate(comm_ops):
    print(f'Same Outp {i+1}: {[file_names[x].replace(".ser","").replace("_output","") for x in item]}')

Same Outp 1: ['pranjal_small_120411']


## Sanity Check

In [26]:
for i,item in enumerate(objects):
    item = objects[i]
    tests = item.testCases
    for j,t in enumerate(tests):
        commands = t.commands
        for k,comm in enumerate(commands):
            if(comm.type=='Sanity'):
                if(comm.output!=0):
                    print(f'Sanity Error in {file_names[i]}:- Test Case {i} Command Num {k}. Value:- {comm.output}')

## Time taken by your program

In [12]:
print(objects[0].time_taken[0].value)  # Total Time taken in complete Execution
print(objects[0].time_taken[1].value)  # Total Time taken in Defragment
print(objects[0].time_taken[2].value)  # Total Time taken in Sanity
print(objects[0].time_taken[3].value)  # Total Time taken in Aloc
print(objects[0].time_taken[4].value)  # Total Time taken in Free

157295300.0
53208100.0
68977700.0
26933900.0
8175600.0


## Compare 2 ser files and see the first point of difference

In [11]:
# Compare where you went wrong

diff = compareTwo(objects[0],objects[1])
# Ouput Format:- 1. Test Case 2. Command number 3. Command Type of first object 
# 4. Cmd Type of 2nd obj. 5. Output of 1st obj. 6. Output of 2nd obj
# prints nothing if output are same

### Object Structure

1. `object[i]` :- The object corresponding to ith files
2. `time_taken`,`total_ops` :- Time taken and total operations done in each of 4 types of operations and total time
3. `testCases` :- list of Objects of class TestCases
4. `num_test`:- Number of test cases in given .ser file

#### Structure of TestCases Class
Contains details of test cases in an single .ser file

1. `size`:- size of test case
2. `commands`:- list of objects of class Command
3. `n_commands`:- Number of commands in this single test case

#### Structure of Command Class
Contains info of each command executed in a test case

1. `type`:- type of operation done
2. `val`:- Input value in test case
3. `output`:- Output given by your program (Note:- A custom output is generated for sanity function where 0 denotes sanity returned true for both blks)
4. `allocSize` & `freeSize`:- Indicates allocBlk,freeBlk size after execution of this command.
5. `corr` & `corr_addr` :- Useful in free operation. `corr_addr` indicates what address was passed in Free function

Enjoy :)